# Layers and Blocks

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)

In [2]:
net(X)

tensor([[ 0.0261,  0.0649,  0.1794,  0.0184,  0.0826,  0.0638,  0.0366, -0.0559,
          0.0492,  0.1147],
        [ 0.0107,  0.0154, -0.0559, -0.0425,  0.0243,  0.1190,  0.0968, -0.0193,
         -0.0347,  0.0876]], grad_fn=<AddmmBackward0>)

In [3]:
class MLP(nn.Module):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Module` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # Hidden layer
        self.out = nn.Linear(256, 10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def forward(self, X):
        # Note here we use the funtional version of ReLU defined in the
        # nn.functional module.
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[ 0.0986,  0.0782,  0.1966, -0.0348, -0.0106, -0.1893,  0.1810, -0.1779,
         -0.0080, -0.1412],
        [ 0.2496,  0.1343,  0.0676, -0.1147,  0.1113, -0.2334,  0.2401, -0.2451,
          0.0473,  0.1068]], grad_fn=<AddmmBackward0>)

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # Here, `module` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            X = block(X)
        return X

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0304, -0.1712,  0.0672,  0.0218, -0.0480,  0.2479, -0.1822,  0.1260,
          0.0061, -0.0151],
        [-0.0384, -0.3154,  0.1240,  0.0389,  0.0626,  0.3717, -0.1487,  0.2666,
         -0.0670, -0.0475]], grad_fn=<AddmmBackward0>)

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # Use the created constant parameters, as well as the `relu` and `mm`
        # functions
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(-0.2122, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

In [10]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0100, grad_fn=<SumBackward0>)

In [11]:
print(chimera)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=20, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=32, bias=True)
      (3): ReLU()
    )
    (linear): Linear(in_features=32, out_features=16, bias=True)
  )
  (1): Linear(in_features=16, out_features=20, bias=True)
  (2): FixedHiddenMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


# Parameter Management

In [12]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), # net0
                    nn.ReLU(),       # net1
                    nn.Linear(8, 1)) # net2
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1080],
        [-0.1310]], grad_fn=<AddmmBackward0>)

In [13]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2611,  0.1965,  0.1938, -0.1419, -0.1306,  0.0673, -0.2636,  0.0022]])), ('bias', tensor([0.0310]))])


In [14]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0310], requires_grad=True)
tensor([0.0310])


In [15]:
net[2].weight.grad == None

True

In [16]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))


In [17]:
print(*[(name, param.shape) for name, param in net.named_parameters()])

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [18]:
net.state_dict()['2.bias'].data

tensor([0.0310])

In [19]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # Nested here
        net.add_module(f'block {i}', block1())
    return net

In [20]:
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.4625],
        [-0.4621]], grad_fn=<AddmmBackward0>)

In [21]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [22]:
rgnet[0][1][0].bias.data

tensor([ 0.4058,  0.4591, -0.0521,  0.4374, -0.0804, -0.2338,  0.0090, -0.3608])

In [25]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

In [26]:
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0038, -0.0055, -0.0031, -0.0180]), tensor(0.))

In [28]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

In [29]:
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [30]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

In [31]:
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0221,  0.6395,  0.1378, -0.6115])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


$$
w \sim \begin{cases}U(5,10) & \text { with probability } \frac{1}{4} \\ 0 & \text { with probability } \frac{1}{2} \\ U(-10,-5) & \text { with probability } \frac{1}{4}\end{cases}
$$

In [32]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

In [35]:
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 6.8246, -0.0000,  9.2734,  6.2784],
        [-5.6095, -0.0000, -5.3549,  7.9503]], grad_fn=<SliceBackward0>)

In [36]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, 10.2734,  7.2784])

In [37]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)

tensor([[-0.3384],
        [-0.3340]], grad_fn=<AddmmBackward0>)

In [38]:
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value, cool!
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# Custom Layers

In [39]:
import torch
from torch import nn
from torch.nn import functional as F


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [40]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))  # call farward()

tensor([-2., -1.,  0.,  1.,  2.])

In [48]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [49]:
print(net)

Sequential(
  (0): Linear(in_features=8, out_features=128, bias=True)
  (1): CenteredLayer()
)


In [51]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(6.0536e-09, grad_fn=<MeanBackward0>)

In [52]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [57]:
linear = MyLinear(5, 3)
print(linear.weight)

Parameter containing:
tensor([[ 1.7717,  1.1718,  0.8007],
        [ 1.7426, -0.1724, -0.0776],
        [ 1.7183,  2.0741,  0.5943],
        [-1.4811,  1.2206,  0.3499],
        [-0.7097,  0.9440,  0.6892]], requires_grad=True)


In [58]:
linear(torch.rand(2, 5))

tensor([[0.0000, 1.5300, 1.9503],
        [1.1062, 3.1275, 2.6102]])

In [59]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[21.1583],
        [14.1512]])

# File I/O

In [61]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, './data/x-file')

In [63]:
x2 = torch.load('./data/x-file')
print(x2)

tensor([0, 1, 2, 3])


In [65]:
y = torch.zeros(4)
torch.save([x, y], './data/x-files')
x2, y2 = torch.load('./data/x-files')
print((x2, y2))

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))


In [67]:
mydict = {'x': x, 'y': y}
torch.save(mydict, './data/mydict')
mydict2 = torch.load('./data/mydict')
print(mydict2)

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}


In [68]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [69]:
torch.save(net.state_dict(), './data/mlp.params')

In [70]:
clone = MLP()
clone.load_state_dict(torch.load('./data/mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [71]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])